<a href="https://colab.research.google.com/github/whelfjhte79/BlindDateApp/blob/main/Face_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip /content/drive/MyDrive/dataset/Humans.zip -d /content/HumanFace/

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import cv2
import matplotlib.pyplot as plt

In [ ]:
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(name=fn,length=len(uploaded[fn])))

In [ ]:
train_df = pd.read_csv('../content/faceData.csv', encoding= 'CP949')
print(train_df.columns.values)

In [ ]:
image = cv2.imread('/content/HumanFace/1 (103).jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)
plt.show()

In [ ]:
image_mask = []
imgPath = "/content/HumanFace/"
for i in os.listdir(imgPath):
    pic = os.path.join(imgPath + "/", i)
    image_mask.append(pic)

In [ ]:
from keras.preprocessing.image import load_img

plt.figure(figsize=(12,8))

img = load_img(mask["image"][8])
plt.imshow(img)
plt.title("With Mask", color = "green", size = 14)
plt.grid(color='#999999', linestyle='-')
plt.show()

In [ ]:
# train + test
# 임시
#face_shape_mapping = { "계란형": 1, "긴얼굴형": 2, "육각형": 3, "마름모형": 4, "둥근형": 5}
eyebrow = { "짙음": 1, "보통": 2, "연함": 3}
skin = {"밝음": 1, "보통": 2, "어두움": 3}

dataset = train_df
#dataset['얼굴형'] = dataset['얼굴형'].map(face_shape_mapping)
dataset['눈썹농도'] = dataset['눈썹농도'].map(eyebrow)
dataset['피부색'] = dataset['피부색'].map(skin)


In [ ]:
train_df.head()

In [ ]:
plt.figure(figsize=(12,8))

img = load_img(mask["image"][8])
img.size


In [ ]:
img = load_img(mask["image"][7])
img.size

In [ ]:
#X_train = image_mask
#Y_train = train_df['얼굴형']
train = pd.DataFrame()
train["image"] = image_mask
train["target"] = train_df['얼굴형']

In [ ]:
import pandas as pd
import numpy as np
import os

from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img


train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   fill_mode = "nearest")

train_generator = train_datagen.flow_from_dataframe(dataframe = train,
                                                    x_col = "image",
                                                    y_col = "target",
                                                    target_size = (150, 150),
                                                    batch_size = 32,
                                                    class_mode = "categorical")

In [ ]:
from keras import layers
from keras import models
from keras import optimizers
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input

vgg19 = VGG19(weights='imagenet',include_top=False,input_shape=(128,128,3))

for layer in vgg19.layers:
    layer.trainable = False
    
model = models.Sequential()
model.add(vgg19)
model.add(layers.Flatten())
model.add(layers.Dense(1,activation='sigmoid'))
model.summary()

In [ ]:
model.compile(loss = "categorical_crossentropy",
             keras.optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0))